In [1]:
import pandas
import numpy
import math
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold, cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
VegiativeIndices = pandas.read_csv('../Datasets/Crop_Yield/Vegitative_Indices/All_Dates_RGB_Indices.csv')
VegiativeIndices.drop(columns=['Unnamed: 0'], axis='columns', inplace=True)
VegiativeIndices['Plot'] = [s.replace('Plot', '') for s in VegiativeIndices['Plot'].values]
classes = VegiativeIndices['Date']
VegiativeIndices.drop(columns=['Date', 'Year'], axis='columns', inplace=True)
VegiativeIndices

,Plot,Red,Green,Blue,BI,GLI,NGRDI,VARI,BGI,ExG,ExR,ExB,ExGR,MGRVI,RGBVI,GRRI,VEG,Range,Row,Mean.Yld.bu.ac
0,10,87.954,108.560,80.241,93.022932,0.126974,0.104858,0.177221,0.739140,48.925,0.052666,0.013649,48.872334,0.207435,0.250905,1.234282,1.272587,35,10,217.233468
1,100,102.090,120.500,89.783,104.886614,0.113491,0.082708,0.138622,0.745087,49.127,0.071792,0.016635,49.055208,0.164292,0.226052,1.180331,1.231917,32,4,221.070525
2,1000,86.488,108.860,81.354,93.003848,0.129364,0.114524,0.196256,0.747327,49.878,0.044175,0.018199,49.833825,0.226082,0.254908,1.258672,1.284584,4,8,200.080613
3,1001,89.941,112.400,85.599,96.695971,0.123045,0.110996,0.192381,0.761557,49.260,0.046945,0.025834,49.213055,0.219290,0.242709,1.249708,1.270470,4,9,204.668282
4,1002,87.312,110.290,82.425,94.129609,0.130261,0.116284,0.199502,0.747348,50.843,0.042663,0.018230,50.800337,0.229466,0.256562,1.263171,1.287633,4,10,223.711903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17311,995,68.891,94.357,58.358,75.398651,0.194532,0.155996,0.242788,0.618481,61.465,0.009433,-0.057109,61.455567,0.304580,0.377829,1.369656,1.447466,5,3,194.246653
17312,996,66.979,91.645,56.253,73.142310,0.195934,0.155500,0.240947,0.613814,60.058,0.009892,-0.059992,60.048108,0.303657,0.380636,1.368265,1.450137,5,4,190.403286
17313,997,65.336,89.657,53.886,71.205972,0.201289,0.156917,0.240547,0.601024,60.092,0.008682,-0.068061,60.083318,0.306292,0.390834,1.372245,1.463174,5,5,269.984577
17314,998,67.874,92.672,57.354,74.127055,0.193565,0.154460,0.240309,0.618892,60.116,0.010792,-0.056799,60.105208,0.301722,0.376194,1.365353,1.444111,5,6,247.938351


In [3]:
# Dates = numpy.unique(VegiativeIndices['Date'].values)

# for date in Dates:
#     filename = '../Datasets/Crop_Yield/Vegitative_Indices/' + str(date) + "_VIndices.csv"
#     df = VegiativeIndices[VegiativeIndices['Date'] == date]

#     df.to_csv(filename)

In [4]:
cv_dates = {
    '6162021': '6152022',
    '6222021': '6232022',
    '6302021': '6282022',
    '7122021': '7132022'
}

In [5]:
VI_2021 = pandas.read_csv('../Datasets/Crop_Yield/Vegitative_Indices/6162021_VIndices.csv')
VI_2022 = pandas.read_csv('../Datasets/Crop_Yield/Vegitative_Indices/6152022_VIndices.csv')
VI_2021.drop(columns=['Unnamed: 0', 'Date', 'Year', 'Range', 'Row'], axis='columns', inplace=True)
VI_2022.drop(columns=['Unnamed: 0', 'Date', 'Year', 'Range', 'Row'], axis='columns', inplace=True)

VI_2021.sort_values(by=['Plot'], inplace=True)
VI_2022.sort_values(by=['Plot'], inplace=True)

In [6]:
VI_2122 = pandas.concat([VI_2021, VI_2022]).reset_index(drop=True)
VI_2122_train_validate = VI_2122[0: math.floor(len(VI_2122) * 0.8)].copy(deep=True).drop(columns=['Plot'], axis='columns')
VI_2122_test = VI_2122[len(VI_2122_train_validate):].copy(deep=True).drop(columns=['Plot'], axis='columns')

In [7]:
scaler = MinMaxScaler()

scaleCols = [
    'Red', 'Green', 'Blue', 'BI', 'GLI', 'NGRDI', 'VARI', 'BGI',
    'ExG', 'ExR', 'ExB', 'ExGR', 'MGRVI', 'RGBVI', 'GRRI', 'VEG',
    'Mean.Yld.bu.ac'
]

VI_2122_train_validate[scaleCols] = scaler.fit_transform(VI_2122_train_validate[scaleCols])
VI_2122_test[scaleCols] = scaler.transform(VI_2122_test[scaleCols])

In [8]:
kmeans = KMeans(n_clusters=5, random_state=41)

VI_2122_train_values = VI_2122_train_validate.drop(columns=['Mean.Yld.bu.ac'])
VI_2122_train_yields = VI_2122_train_validate['Mean.Yld.bu.ac']

classes_2122 = kmeans.fit(VI_2122_train_values).labels_


In [9]:
gkf = GroupKFold(n_splits=4)

LRmodel = LinearRegression()
RFmodel = RandomForestRegressor(random_state=41)

folds = gkf.split(VI_2122_train_values, VI_2122_train_yields, classes_2122)
train_indices, test_indices = [list(train) for train in zip(*folds)]
VI_cv = [*zip(train_indices,test_indices)]

LR_RMSE = -cross_val_score(LRmodel, VI_2122_train_values, VI_2122_train_yields, cv=VI_cv, scoring='neg_root_mean_squared_error')
LR_r2 = cross_val_score(LRmodel, VI_2122_train_values, VI_2122_train_yields, cv=VI_cv, scoring='r2')

RF_RMSE = -cross_val_score(RFmodel, VI_2122_train_values, VI_2122_train_yields, cv=VI_cv, scoring='neg_root_mean_squared_error')
RF_r2 = cross_val_score(RFmodel, VI_2122_train_values, VI_2122_train_yields, cv=VI_cv, scoring='r2')

print("LR CV RMSE Scores: ", LR_RMSE)
print("LR CV Average RMSE Score: ", numpy.average(LR_RMSE))

print("LR CV R^2 Scores: ", LR_r2)
print("LR CV Average R^2 Score: ", numpy.average(LR_r2))

print("--------------------------------")
print("RF CV RMSE Scores: ", RF_RMSE)
print("RF CV Average RMSE Score: ", numpy.average(RF_RMSE))

print("RF CV R^2 Scores: ", RF_r2)
print("RF CV Average R^2 Score: ", numpy.average(RF_r2))

LR CV RMSE Scores:  [0.18053085 0.17043623 0.14175188 0.28117753]
LR CV Average RMSE Score:  0.1934741211738365
LR CV R^2 Scores:  [-0.0319433  -0.1046445  -5.54986565 -0.82625381]
LR CV Average R^2 Score:  -1.628176817773282
--------------------------------
RF CV RMSE Scores:  [0.1668982  0.17710872 0.06852591 0.13637111]
RF CV Average RMSE Score:  0.13722598547067932
RF CV R^2 Scores:  [ 0.11802496 -0.19283016 -0.53068     0.57041941]
RF CV Average R^2 Score:  -0.008766447402499433


In [10]:
LRmodel.fit(VI_2122_train_values, VI_2122_train_yields)
RFmodel.fit(VI_2122_train_values, VI_2122_train_yields)

X_test = VI_2122_test.drop(columns=['Mean.Yld.bu.ac'])
y_test = VI_2122_test['Mean.Yld.bu.ac']

LRr2 = LRmodel.score(X_test, y_test)
RFr2 = RFmodel.score(X_test, y_test)

LRPredictions = LRmodel.predict(X_test)
RFPredictions = RFmodel.predict(X_test)

LRRMSE = root_mean_squared_error(y_test, LRPredictions)
RFRMSE = root_mean_squared_error(y_test, RFPredictions)


print("LR Model R^2 Score: ", LRr2)
print("LR Model RMSE Score: ", LRRMSE)
print("RF Model R^2 Score: ", RFr2)
print("RF Model RMSE Score: ", RFRMSE)

LR Model R^2 Score:  -5.42985060907073
LR Model RMSE Score:  0.29256302201322093
RF Model R^2 Score:  -0.3387217592962155
RF Model RMSE Score:  0.13349473981441878
